In [2]:
import numpy as np,pandas as pd,json,sqlite3,requests, os
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
from dotenv import load_dotenv
load_dotenv()
db_path = os.environ.get('dsa_db')
char_path = os.environ.get('char_data')
game_path = os.environ.get('game_data')
with open(char_path, 'r') as char_data:
    chars_json = json.load(char_data)

In [111]:
def fix_gear(dataf):
    g = 1
    length = dataf.shape[0]
    for i in range(0,length):
        if i % 6 == 0 and i != 0: g+=1
        dataf.at[i,'Gear'] = g
    return dataf


In [112]:
jack_data = chars_json['150']['GearTiers']

In [113]:
jack_df = pd.json_normalize(jack_data,record_path=['GearTierSlot'],errors='ignore')

In [114]:
jack_df = fix_gear(jack_df)

In [115]:
jack_df['Toon_ID'] = 150

In [82]:
jack_df.index
jack_df.columns

RangeIndex(start=0, stop=48, step=1)

In [116]:
jack_df = jack_df[['Toon_ID', 'Gear', 'Offense', 'Defense', 'Health', 'Recovery', 'CritChance', 'CritPower',
       'AttackSpeed','Speed', 'Potency', 'Tenacity',
       'Accuracy', 'Evasion','UnitLevel','ItemID']]
jack_df

,Toon_ID,Gear,Offense,Defense,Health,Recovery,CritChance,CritPower,AttackSpeed,Speed,Potency,Tenacity,Accuracy,Evasion,UnitLevel,ItemID
0,150,1.0,True,False,False,False,False,False,False,False,False,False,False,False,5,5800404643593210
1,150,1.0,False,True,False,False,False,False,False,False,False,False,False,False,1,6291390382388483
2,150,1.0,False,False,True,False,False,False,False,False,False,False,False,False,8,3179470540328821
3,150,1.0,True,False,False,False,False,False,False,False,False,False,False,False,5,5800404643593210
4,150,1.0,False,False,True,False,False,False,False,False,False,False,False,False,8,3179470540328821
5,150,1.0,False,False,True,False,False,False,False,False,False,False,False,False,8,3179470540328821
6,150,2.0,True,False,False,False,False,False,False,False,False,False,False,False,12,8331552497003191
7,150,2.0,False,True,False,False,False,False,False,False,False,False,False,False,9,5274591237803763
8,150,2.0,False,False,True,False,False,False,False,False,False,False,False,False,15,1949330146132045
9,150,2.0,True,False,False,False,False,False,False,False,False,False,False,False,12,8331552497003191


In [117]:
with sqlite3.connect(db_path) as conn:
    conn.execute('DROP TABLE IF EXISTS gear_items')
    for toon in chars_json:
        toon_data = chars_json[toon]['GearTiers']
        toon_df = pd.json_normalize(toon_data,record_path=['GearTierSlot'],errors='ignore')
        toon_df = fix_gear(toon_df)
        toon_df['Toon_ID'] = chars_json[toon]['ID']
        #if toon_df.shape[0] == 48:
        try:
            toon_df = toon_df[['Toon_ID', 'Gear', 'Offense', 'Defense', 'Health', 'Recovery', 'CritChance', 'CritPower',
        'AttackSpeed','Speed', 'Potency', 'Tenacity',
        'Accuracy', 'Evasion','UnitLevel','ItemID']]
            toon_df.to_sql('gear_items',conn,if_exists='append',index=False)
        except:
            continue

In [128]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql('SELECT * FROM toons_categories', conn)
    df['category_name'] = df['category_name'].apply(lambda x: x.replace('UNIT_CATEGORY_','').replace('_',' ').title())
    df.to_sql('toons_categories',conn, if_exists='replace',index = False)